In [1]:
# import necessary packages and utils

import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models, transforms, datasets
from torch.utils.data import Dataset, DataLoader, Subset

import PIL.Image as Image
from tqdm import tqdm
import pickle

from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error as mse
from sklearn import linear_model
from sklearn.model_selection import StratifiedShuffleSplit

from models.vgg16 import *

In [2]:
# config   

data_dir = "../data/petfinder-pawpularity-score/"

num_classes = 1 # for regression
batch_size = 16
learning_rate = 1e-3

# model_name = vgg16_bn(pretrained=True) # baseline model

In [3]:
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

display(device)

device(type='cuda')

In [4]:
# Image Dataset - might need modification later for covariates idk yet

class PetDataset(Dataset):
    def __init__(self, image_filepaths, targets, transform=None):
        self.image_filepaths = image_filepaths
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.image_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.image_filepaths[idx]
        
        with open(image_filepath, 'rb') as f:
            image = Image.open(f)
            image_rgb = image.convert('RGB')

        if self.transform is not None:
            image = self.transform(image)

        target = torch.tensor(self.targets[idx])
        
        return image, target

In [5]:
# load cvs data into dataframes

def get_dataframe(data_dir, is_train=True):
    
    if is_train:
        image_dir = os.path.join(data_dir, 'train')
        file_path = os.path.join(data_dir, 'train.csv')
    else:
        image_dir = os.path.join(data_dir, 'test')
        file_path = os.path.join(data_dir, 'test.csv')
    
    df = pd.read_csv(file_path)

    # set image filepath
    df['img_file_path'] = df['Id'].apply(lambda x: os.path.join(image_dir, f'{x}.jpg'))
    
    return df

train_df = get_dataframe(data_dir, is_train=True)
test_df = get_dataframe(data_dir, is_train=False)

In [6]:
# get DataLoader

def load_data(batch_size, is_train=True, use_subset=False):
    """
    return the train dataloader
    """
    
    if is_train:
        df = get_dataframe(data_dir, is_train=True)
        images = np.array(df['img_file_path'])
        targets = np.array(df['Pawpularity'])
    else:
        df = get_dataframe(data_dir, is_train=False)
        images = np.array(df['img_file_path'])
        targets = np.zeros_like(images)
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
    ])
    
    dataset = PetDataset(image_filepaths=images, targets=targets, transform=transform)
    
    subse_ind = list(range(500))
    
    data_subset = Subset(dataset, subse_ind)

    # data loader
    data_loader = DataLoader(dataset=data_subset if use_subset else dataset, 
                                batch_size=batch_size,
                                shuffle=True)
    
    return data_loader

train_loader = load_data(batch_size, is_train=True, use_subset=True)
test_loader = load_data(batch_size, is_train=False)

In [7]:
# get model, loss function and optimizer

def initialize_model(model, learning_rate, num_classes):
    """
    initialize the model
    define loss function and optimizer and move data to gpu if available
    
    return:
        model, loss function(criterion), optimizer
    """
    
    if isinstance(model, VGG):
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    
    model = model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    return model, criterion, optimizer

model_name = vgg16_bn(pretrained=True)

model, criterion, optimizer = initialize_model(model_name, learning_rate, num_classes)

In [9]:
# plotting loss curve

def make_plots(step_hist, loss_hist):
    plt.plot(step_hist, loss_hist)
    plt.xlabel('train_iterations')
    plt.ylabel('Loss')
    plt.title(experiment_name)
    plt.show()
    plt.savefig(experiment_name)
    plt.clf()

In [10]:
# train

experiment_name = "simple_run"

def calc_rmse(y_pred, y_true):
    return np.sqrt(((y_pred - y_true) ** 2).mean())


def train(train_loader, model, criterion, optimizer):
    """
    Move data to GPU memory
    Also plot the loss function and save it in `Figures/`
    Trained model is saved as `cnn.ckpt`
    """
    model.train()
    
    # for each training sample
    loss_hist = []
    step_hist = []
    for i, (images, label) in (enumerate(train_loader)):

        train_pred = list()
        train_true = list()

        # move to gpu if available
        images = images.to(device).float()
        label = label.to(device).float()

        # forward pass
        out = model(images)
        loss = torch.sqrt(criterion(out, label))

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 50 == 0:
            train_true += label.cpu().detach().numpy().tolist()
            train_pred += out.cpu().detach().numpy().tolist()

            train_rmse = calc_rmse(np.array(train_pred), np.array(train_true))
            
            step_hist.append(i+1)
            loss_hist.append(train_rmse)
            print('Iteration: {}, Train rmse: {}'.format(i+1, train_rmse))
            
    # plot
    make_plots(step_hist, loss_hist)

    torch.save(model.state_dict(), experiment_name+'.ckpt')

In [11]:
train(train_loader, model, criterion, optimizer)

C:\Users\genglinliu\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 4.00 GiB total capacity; 2.47 GiB already allocated; 258.29 MiB free; 2.54 GiB reserved in total by PyTorch)

In [ ]:
# eval

def evaluation(test_loader, model):
    model.eval() 
    print('Making predictions...')
    test_pred = []    
    
    with torch.no_grad():
        for test_images, test_labels in test_loader:
            test_images = test_images.to(device).float()
            # forward
            out = model(test_images)
            test_pred += out.cpu().numpy().tolist()

        # write to file
        output = pd.DataFrame({"Id": test_df['Id'], "Pawpularity": test_pred})
        output.to_csv('submission.csv', index = False)

        # check output
        output_df = pd.read_csv('submission.csv')

        return output_df

evaluation(test_loader, model)